In [1]:
import numpy as np
import pywt
import soundfile
from tqdm import tqdm

In [3]:
def median_abs_derivation(arr):
    """ Median Absolute Deviation: a "Robust" version of standard deviation.
        Indices variability of the sample.
    """
    arr = np.ma.array(arr).compressed()
    med = np.median(arr)
    return np.median(np.abs(arr - med))


class AudioDeNoise:
    """
    Class to de-noise the audio signal. The audio file is read in chunks and processed,
    cleaned and appended to the output file..

    It can de-noise multiple channels, any sized file, formats supported by soundfile

    Wavelets used ::
        Daubechies 4 : db4
        Level : decided by pyWavelets

    Attributes
    ----------
    __inputFile : str
        name of the input audio file
    """

    def __init__(self, inputFile):
        self.__inputFile = inputFile
        self.__noiseProfile = None

    def deNoise(self, outputFile):
        """
        De-noising function that reads the audio signal in chunks and processes
        and writes to the output file efficiently.

        VISU Shrink is used to generate the noise threshold

        Parameters
        ----------
        outputFile : str
            de-noised file name

        """
        info = soundfile.info(self.__inputFile)  # getting info of the audio
        rate = info.samplerate

        with soundfile.SoundFile(outputFile, "w", samplerate=rate, channels=info.channels) as of:
            for block in tqdm(soundfile.blocks(self.__inputFile, int(rate * info.duration * 0.10))):
                coefficients = pywt.wavedec(block, 'db4', mode='per', level=2)

                #  getting variance of the input signal
                sigma = median_abs_derivation(coefficients[- 1])

                # VISU Shrink thresholding by applying the universal threshold proposed by Donoho and Johnstone
                thresh = sigma * np.sqrt(2 * np.log(len(block)))

                # thresholding using the noise threshold generated
                coefficients[1:] = (pywt.threshold(i, value=thresh, mode='soft') for i in coefficients[1:])

                # getting the clean signal as in original form and writing to the file
                clean = pywt.waverec(coefficients, 'db4', mode='per')
                of.write(clean)

#denoising a file
audioDenoiser = AudioDeNoise("/Users/lizawahi/Downloads/pretrained.mp3")
audioDenoiser.deNoise("denoised_pretrained.wav")
#audioDenoiser.generateNoiseProfile("/Users/lizawahi/Downloads/pretrained.mp3")

0it [00:00, ?it/s][src/libmpg123/layer3.c:INT123_do_layer3():1773] error: part2_3_length (1376) too large for available bit count (1352)
[src/libmpg123/layer3.c:INT123_do_layer3():1773] error: part2_3_length (992) too large for available bit count (936)
[src/libmpg123/layer3.c:INT123_do_layer3():1773] error: part2_3_length (1024) too large for available bit count (936)
[src/libmpg123/layer3.c:INT123_do_layer3():1773] error: part2_3_length (1376) too large for available bit count (1352)
8it [00:00, 79.53it/s][src/libmpg123/layer3.c:INT123_do_layer3():1773] error: part2_3_length (992) too large for available bit count (936)
10it [00:00, 91.02it/s]
